In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='LearnDataFrame')


Save cars.csv to /predix/ for later usage.

```
year,make,model,comment,blank
2012,Tesla,S,No comment,
,,,,
1997,Ford,E350,Go get one now they are going fast,
2015,Chevy,Volt,,
2017,Audi,QQ,,
2018,Citroen,,,
2012,Tesla,S,No comment,
,,,,
1997,Ford,E350,Go get one now they are going fast,
2015,Chevy,Volt,,
2017,Audi,QQ,,
2018,Citroen,,,
2012,Tesla,S,No comment,
1997,Ford,E350,Go get one now they are going fast,
2017,Audi,QQ,,
2018,Citroen,,,
2012,Tesla,S,No comment,
2015,Chevy,Volt,,
2017,Audi,QQ,,
2018,Citroen,,,
```

In [5]:
data = ['Hello' , 'I' , 'AM', 'Ankit ', 'Gupta']
rdd = sc.parallelize(data)
rdd.take(3)  # just print 3, safe choice when you have infi lines
# rdd.collect()  # print all


['Hello', 'I', 'AM']

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
# https://www.analyticsvidhya.com/blog/2016/09/comprehensive-introduction-to-apache-spark-rdds-dataframes-using-pyspark/
# train = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('train.csv')

# https://github.com/databricks/spark-csv#python-api

cars = sqlContext.read.format('csv').options(header='true', inferschema='true').load('cars.csv') # load CSV

cars.printSchema()  # list headers as tree

print(cars.count())  # num of rows in CSV
print('\n')

print(cars.take(2))  # print first 2 rows as list
print('\n')

cars.show()  # by default show first 20 lines as table



root
 |-- year: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- blank: string (nullable = true)

20


[Row(year=2012, make='Tesla', model='S', comment='No comment', blank=None), Row(year=None, make=None, model=None, comment=None, blank=None)]


+----+-------+-----+--------------------+-----+
|year|   make|model|             comment|blank|
+----+-------+-----+--------------------+-----+
|2012|  Tesla|    S|          No comment| null|
|null|   null| null|                null| null|
|1997|   Ford| E350|Go get one now th...| null|
|2015|  Chevy| Volt|                null| null|
|2017|   Audi|   QQ|                null| null|
|2018|Citroen| null|                null| null|
|2012|  Tesla|    S|          No comment| null|
|null|   null| null|                null| null|
|1997|   Ford| E350|Go get one now th...| null|
|2015|  Chevy| Volt|                null| null|
|2017|   Audi|   QQ|                

In [7]:
# By default, drop() method will drop a row if it contains any null value.
# We can also pass ‘all” to drop a row only if all its values are null.
cars.na.drop('any').show()
cars.na.drop("all").show()

# na.fill() will replace null with the same type int by 0 and str by '-'
cars.na.fill(0).na.fill('-').show()


+----+----+-----+-------+-----+
|year|make|model|comment|blank|
+----+----+-----+-------+-----+
+----+----+-----+-------+-----+

+----+-------+-----+--------------------+-----+
|year|   make|model|             comment|blank|
+----+-------+-----+--------------------+-----+
|2012|  Tesla|    S|          No comment| null|
|1997|   Ford| E350|Go get one now th...| null|
|2015|  Chevy| Volt|                null| null|
|2017|   Audi|   QQ|                null| null|
|2018|Citroen| null|                null| null|
|2012|  Tesla|    S|          No comment| null|
|1997|   Ford| E350|Go get one now th...| null|
|2015|  Chevy| Volt|                null| null|
|2017|   Audi|   QQ|                null| null|
|2018|Citroen| null|                null| null|
|2012|  Tesla|    S|          No comment| null|
|1997|   Ford| E350|Go get one now th...| null|
|2017|   Audi|   QQ|                null| null|
|2018|Citroen| null|                null| null|
|2012|  Tesla|    S|          No comment| null|
|2015| 

In [8]:
cars.describe().show()  # show statistics of DataFrame


+-------+------------------+-----+-----+--------------------+-----+
|summary|              year| make|model|             comment|blank|
+-------+------------------+-----+-----+--------------------+-----+
|  count|                18|   18|   14|                   7|    0|
|   mean|2012.4444444444443| null| null|                null| null|
| stddev|    7.445663737823| null| null|                null| null|
|    min|              1997| Audi| E350|Go get one now th...| null|
|    max|              2018|Tesla| Volt|          No comment| null|
+-------+------------------+-----+-----+--------------------+-----+



In [9]:
cars.select('year', 'model').collect()  # get column by select


[Row(year=2012, model='S'),
 Row(year=None, model=None),
 Row(year=1997, model='E350'),
 Row(year=2015, model='Volt'),
 Row(year=2017, model='QQ'),
 Row(year=2018, model=None),
 Row(year=2012, model='S'),
 Row(year=None, model=None),
 Row(year=1997, model='E350'),
 Row(year=2015, model='Volt'),
 Row(year=2017, model='QQ'),
 Row(year=2018, model=None),
 Row(year=2012, model='S'),
 Row(year=1997, model='E350'),
 Row(year=2017, model='QQ'),
 Row(year=2018, model=None),
 Row(year=2012, model='S'),
 Row(year=2015, model='Volt'),
 Row(year=2017, model='QQ'),
 Row(year=2018, model=None)]

In [10]:
cars.select('year').distinct().count()  # column select

6

In [11]:
cars.select('year').subtract(cars.select('year')).show()  # column subtract

+----+
|year|
+----+
+----+



# SparkSession - a new entry point
https://docs.databricks.com/spark/latest/gentle-introduction/sparksession.html

We have been getting a lot of questions about thre relationship between SparkContext, SQLContext, and HiveContext in Spark 1.x. It was really strange to have "HiveContext" as an entry point when people want to use the DataFrame API. In Spark 2.0, we are introducing SparkSession, a new entry point that subsumes SQLContext and HiveContext. For backward compatibiilty, the two are preserved. SparkSession has many features, and here we demonstrate some of the more important ones.

To read the companion blog post, click here: https://databricks.com/blog/2016/05/11/spark-2-0-technical-preview-easier-faster-and-smarter.html



# Creating a SparkSession

A SparkSession can be created using a builder pattern. The builder will automatically reuse an existing SparkContext if one exists; and create a SparkContext if it does not exist. Configuration options set in the builder are automatically propagated over to Spark and Hadoop during I/O.

<!-- -->
// A SparkSession can be created using a builder pattern
import org.apache.spark.sql.SparkSession
val sparkSession = SparkSession.builder
  .master("local")
  .appName("my-spark-app")
  .config("spark.some.config.option", "config-value")
  .getOrCreate()
  
https://databricks.com/blog/2016/08/15/how-to-use-sparksession-in-apache-spark-2-0.html

`from pyspark.sql import SparkSession`


# stringIndexer

make a string to a number then you can select it for ML calculation as a feature.
https://spark.apache.org/docs/2.2.0/ml-features.html#stringindexer


In [12]:
from pyspark.ml.feature import StringIndexer

from pyspark.sql import SparkSession
spark = SparkSession.builder. \
        appName('LearnDataFrame') \
        .getOrCreate()
        
df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])
df.show()

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

+---+--------+
| id|category|
+---+--------+
|  0|       a|
|  1|       b|
|  2|       c|
|  3|       a|
|  4|       a|
|  5|       c|
+---+--------+

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+



In [4]:
# https://stackoverflow.com/questions/34681534/spark-ml-stringindexer-handling-unseen-labels

makeIndexer = StringIndexer(inputCol="make", outputCol="makeIndexed", \
                            handleInvalid="skip") # options are "keep", "error" or "skip" to handle null

# otherwise below error returned.
# Caused by: org.apache.spark.SparkException: StringIndexer encountered NULL value. To handle or skip NULLS, 
# try setting StringIndexer.handleInvalid.

makeIndexed = makeIndexer.fit(cars).transform(cars)
makeIndexed.show()

makeIndexed.select('year', 'makeIndexed').toDF('label', 'features').show()  # change column name


+----+-------+-----+--------------------+-----+-----------+
|year|   make|model|             comment|blank|makeIndexed|
+----+-------+-----+--------------------+-----+-----------+
|2012|  Tesla|    S|          No comment| null|        1.0|
|1997|   Ford| E350|Go get one now th...| null|        4.0|
|2015|  Chevy| Volt|                null| null|        3.0|
|2017|   Audi|   QQ|                null| null|        0.0|
|2018|Citroen| null|                null| null|        2.0|
|2012|  Tesla|    S|          No comment| null|        1.0|
|1997|   Ford| E350|Go get one now th...| null|        4.0|
|2015|  Chevy| Volt|                null| null|        3.0|
|2017|   Audi|   QQ|                null| null|        0.0|
|2018|Citroen| null|                null| null|        2.0|
|2012|  Tesla|    S|          No comment| null|        1.0|
|1997|   Ford| E350|Go get one now th...| null|        4.0|
|2017|   Audi|   QQ|                null| null|        0.0|
|2018|Citroen| null|                null